In [1]:
# Detect Behavior with Sensor Data – CNN + Bi-LSTM + Demographics
# ------------------------------------------------------------------
# This is a minimally-intrusive revision of your original notebook.
# The only functional addition is that the seven demographic/anthro-
# pometric columns from train_demographics.csv are merged onto every
# row of the sensor frame and treated as extra numeric channels.
# Nothing else in the pipeline changes, so you can reuse previous
# hyper-parameters and checkpoints if desired.

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (
    Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization,
    LSTM, Bidirectional, GlobalAveragePooling1D
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf
import polars as pl
#import kaggle_evaluation.cmi_inference_server  # noqa: F401   | Kaggle runner hook

print("Imports loaded")

# ------------------------------------------------------------------
# 1.  LOAD TRAIN SENSOR DATA + DEMOGRAPHICS
# ------------------------------------------------------------------
print("Loading sensor dataset …")
root = '/Users/ashhadulislam/projects/general_data/CMI/ Detect Behavior with Sensor Data/cmi-detect-behavior-with-sensor-data/'

df = pd.read_csv(f"{root}/train.csv")
print(f"Loaded {len(df):,} rows of sensor frames")

# --- NEW: merge participant demographics on the key `subject` --------
print("Merging demographic attributes …")
demographics = pd.read_csv(f"{root}/train_demographics.csv")
df = df.merge(demographics, on="subject", how="left")



Imports loaded
Loading sensor dataset …
Loaded 574,945 rows of sensor frames
Merging demographic attributes …


In [2]:
# ------------------------------------------------------------------
# 2.  LABEL-ENCODE GESTURE TARGET
# ------------------------------------------------------------------
label_encoder = LabelEncoder()
df["gesture"] = label_encoder.fit_transform(df["gesture"].astype(str))
np.save("gesture_classes_multiclass.npy", label_encoder.classes_)

print("Gesture label mapping:")
for idx, lab in enumerate(label_encoder.classes_):
    print(f"  {idx}: {lab}")


Gesture label mapping:
  0: Above ear - pull hair
  1: Cheek - pinch skin
  2: Drink from bottle/cup
  3: Eyebrow - pull hair
  4: Eyelash - pull hair
  5: Feel around in tray and pull out an object
  6: Forehead - pull hairline
  7: Forehead - scratch
  8: Glasses on/off
  9: Neck - pinch skin
  10: Neck - scratch
  11: Pinch knee/leg skin
  12: Pull air toward your face
  13: Scratch knee/leg skin
  14: Text on phone
  15: Wave hello
  16: Write name in air
  17: Write name on leg


In [3]:
label_encoder.classes_

array(['Above ear - pull hair', 'Cheek - pinch skin',
       'Drink from bottle/cup', 'Eyebrow - pull hair',
       'Eyelash - pull hair',
       'Feel around in tray and pull out an object',
       'Forehead - pull hairline', 'Forehead - scratch', 'Glasses on/off',
       'Neck - pinch skin', 'Neck - scratch', 'Pinch knee/leg skin',
       'Pull air toward your face', 'Scratch knee/leg skin',
       'Text on phone', 'Wave hello', 'Write name in air',
       'Write name on leg'], dtype=object)

In [4]:

# ------------------------------------------------------------------
# 3.  FEATURE LIST CONSTRUCTION
# ------------------------------------------------------------------
# Optionally skip thermal/TOF values → set to False to use them.

drop_thermal_and_tof = False

excluded_cols = {
    "gesture", "sequence_type", "behavior", "orientation",  # train-only targets
    "row_id", "subject", "phase",                            # meta
    "sequence_id", "sequence_counter"                         # ids
}

thermal_tof_cols = [c for c in df.columns if c.startswith(("thm_", "tof_"))]

if drop_thermal_and_tof:
    excluded_cols.update(thermal_tof_cols)
    print(f"Ignoring {len(thermal_tof_cols)} thermopile/TOF channels → set drop_thermal_and_tof=False to use them.")

# --- NEW: demographic numeric columns --------------------------------
demographic_cols = [
    "adult_child", "age", "sex", "handedness",
    "height_cm", "shoulder_to_wrist_cm", "elbow_to_wrist_cm",
]

# Combine sensor + demographic feature list
feature_cols = [c for c in df.columns if c not in excluded_cols]
print(f"Using {len(feature_cols)} feature columns for training, including demographics:")
print(sorted(feature_cols)[:15], "…")

# Check missing values
nan_total = df[feature_cols].isna().sum().sum()
print(f"Total NaNs inside feature matrix: {nan_total:,}")

Using 339 feature columns for training, including demographics:
['acc_x', 'acc_y', 'acc_z', 'adult_child', 'age', 'elbow_to_wrist_cm', 'handedness', 'height_cm', 'rot_w', 'rot_x', 'rot_y', 'rot_z', 'sex', 'shoulder_to_wrist_cm', 'thm_1'] …
Total NaNs inside feature matrix: 3,597,807


In [5]:
# ------------------------------------------------------------------
# 4.  SEQUENCE BUILDING HELPERS
# ------------------------------------------------------------------

def preprocess_sequence(df_seq: pd.DataFrame, feature_columns: list[str]) -> np.ndarray:
    """Fill→scale a *single* sequence dataframe and return float32 numpy."""
    data = df_seq[feature_columns].copy()
    data = data.ffill().bfill().fillna(0.0)
    scaled = StandardScaler().fit_transform(data)   # per-sequence scaler (unchanged)
    return scaled.astype("float32")

print("Constructing padded tensor dataset …")
seq_groups = df.groupby("sequence_id")

X, seq_lengths = [], []
for i, (_, seq) in enumerate(seq_groups):
    if i and i % 500 == 0:
        print(f"  processed {i} sequences …")
    arr = preprocess_sequence(seq, feature_cols)
    X.append(arr)
    seq_lengths.append(arr.shape[0])

pad_len = int(np.percentile(seq_lengths, 90))
print(f"90th-percentile length = {pad_len} → fixed pad length chosen")
np.save("sequence_maxlen.npy", pad_len)



Constructing padded tensor dataset …
  processed 500 sequences …
  processed 1000 sequences …
  processed 1500 sequences …
  processed 2000 sequences …
  processed 2500 sequences …
  processed 3000 sequences …
  processed 3500 sequences …
  processed 4000 sequences …
  processed 4500 sequences …
  processed 5000 sequences …
  processed 5500 sequences …
  processed 6000 sequences …
  processed 6500 sequences …
  processed 7000 sequences …
  processed 7500 sequences …
  processed 8000 sequences …
90th-percentile length = 103 → fixed pad length chosen


In [6]:
pad_len

103

In [7]:
X = pad_sequences(X, maxlen=pad_len, dtype="float32", padding="post", truncating="post")

y = seq_groups["gesture"].first().values
num_classes = len(np.unique(y))
y = to_categorical(y, num_classes=num_classes)

# ------------------------------------------------------------------
# 5.  TRAIN/VAL SPLIT & MODEL
# ------------------------------------------------------------------
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)
print(X.shape,y.shape,X_train.shape,y_train.shape,X_val.shape,y_val.shape)

(8151, 103, 339) (8151, 18) (6520, 103, 339) (6520, 18) (1631, 103, 339) (1631, 18)


In [14]:
y_train.shape

(6520, 18)

In [9]:
print("Building CNN-BiLSTM model …")
model = Sequential([
    Conv1D(64, 3, activation="relu", input_shape=(pad_len, X_train.shape[2])),
    BatchNormalization(),
    Conv1D(64, 3, activation="relu"),
    MaxPooling1D(2),
    Dropout(0.30),

    Conv1D(128, 5, activation="relu"),
    BatchNormalization(),
    Conv1D(128, 5, activation="relu"),
    MaxPooling1D(2),
    Dropout(0.30),

    Conv1D(256, 7, activation="relu"),
    BatchNormalization(),
    Conv1D(256, 7, activation="relu"),
    MaxPooling1D(2),
    Dropout(0.40),

    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.40),

    GlobalAveragePooling1D(),

    Dense(512, activation="relu"),
    BatchNormalization(),
    Dropout(0.50),
    Dense(256, activation="relu"),
    Dropout(0.30),

    Dense(num_classes, activation="softmax"),
])


Building CNN-BiLSTM model …


/Users/ashhadulislam/miniconda3/envs/py310/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.compile(optimizer=Adam(1e-3), loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

print("Training …")
callbacks = [
    ReduceLROnPlateau(patience=4, factor=0.5, verbose=1),
    EarlyStopping(patience=8, restore_best_weights=True, verbose=1)
]
model.fit(X_train, y_train, epochs=60, batch_size=128,
          validation_data=(X_val, y_val), callbacks=callbacks)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 101, 64)        │        65,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 101, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 99, 64)         │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 49, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 49, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 45, 128)        │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 45, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 41, 128)        │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 20, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 20, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 14, 256)        │       229,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 14, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 8, 256)         │       459,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 4, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 4, 256)         │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 4, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 1,554,898 (5.93 MB)

 Trainable params: 1,552,978 (5.92 MB)

 Non-trainable params: 1,920 (7.50 KB)

Training …
Epoch 1/60
51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - accuracy: 0.0761 - loss: 3.5985 - val_accuracy: 0.0938 - val_loss: 2.8293 - learning_rate: 0.0010
Epoch 2/60
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - accuracy: 0.1427 - loss: 2.8639 - val_accuracy: 0.1907 - val_loss: 2.6611 - learning_rate: 0.0010
Epoch 3/60
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 70ms/step - accuracy: 0.1984 - loss: 2.5126 - val_accuracy: 0.2391 - val_loss: 2.4543 - learning_rate: 0.0010
Epoch 4/60
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step - accuracy: 0.2375 - loss: 2.2902 - val_accuracy: 0.2679 - val_loss: 2.2539 - learning_rate: 0.0010
Epoch 5/60
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - accuracy: 0.2882 - loss: 2.0941 - val_accuracy: 0.2826 - val_loss: 2.0569 - learning_rate: 0.0010
Epoch 6/60
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.3063 - loss: 1.9775 - val_accuracy: 0.3348 - val_loss: 1.8816 - learning_rate: 0.0010
Epoch 7/60
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - accuracy: 0.3414 - loss: 1.846

In [11]:
model.save("models/gesture_cnn_model.h5")
print("Training complete; model saved → gesture_cnn_model.h5")


Training complete; model saved → gesture_cnn_model.h5


In [13]:
# ------------------------------------------------------------------
# 6.  LOCAL VALIDATION METRIC
# ------------------------------------------------------------------
print("Computing validation hierarchical-F1 …")
from cmi_2025_metric_copy_for_import import CompetitionMetric  # local helper

probs_val = model.predict(X_val, verbose=0)
labels_val_pred = np.argmax(probs_val, axis=1)
labels_val_true = np.argmax(y_val, axis=1)

cls = np.load("gesture_classes_multiclass.npy", allow_pickle=True)
val_pred_df = pd.DataFrame({"gesture": [cls[i] for i in labels_val_pred]})
val_true_df = pd.DataFrame({"gesture": [cls[i] for i in labels_val_true]})

metric = CompetitionMetric()
score = metric.calculate_hierarchical_f1(val_true_df, val_pred_df)
print(f"Estimated public-LB score on held-out fold: {score:.4f}")

Computing validation hierarchical-F1 …
Estimated public-LB score on held-out fold: 0.6529


In [ ]:
# ------------------------------------------------------------------
# 7.  INFERENCE HELPER
# ------------------------------------------------------------------

def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    """Kaggle inference signature: returns predicted gesture string."""
    seq_df = sequence.to_pandas()
    demo_df = demographics.to_pandas()
    seq_df = seq_df.merge(demo_df, on="subject", how="left")

    arr = preprocess_sequence(seq_df, feature_cols)
    maxlen = int(np.load("sequence_maxlen.npy"))
    padded = pad_sequences([arr], maxlen=maxlen, dtype="float32", padding="post", truncating="post")

    mdl = load_model("gesture_cnn_model.h5")
    probs = mdl.predict(padded, verbose=0)
    idx = int(np.argmax(probs, axis=1)[0])
    classes = np.load("gesture_classes_multiclass.npy", allow_pickle=True)
    return str(classes[idx])

In [ ]:
import kaggle_evaluation.cmi_inference_server  # noqa: F401   | Kaggle runner hook


# Launch inference server
inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        data_paths=(
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
        )
    )

In [ ]:
predict